<a href="https://colab.research.google.com/github/zoltanzack/CS3TM20/blob/master/CW_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 

# **Steps outline**
1. Download your data set by inputting your student number.
2. Process your text data, extract features, convert them into vectors
3. Modeling, train models on the data set (select model, tune different parameters)
4. Process your text data, extract features, convert them into vectors
5. Analysis and discussions

# Step 1: Load Dataset


In [5]:
# Imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [58]:
# Import dataset
from sklearn.datasets import fetch_20newsgroups

# Possible categories
#dataset_categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
dataset_categories = ['comp.graphics', 'alt.atheism']

# Training and testing dataset
  # fetch_20newsgroups function is a data fetching / caching functions that downloads the data archive from the original `20 newsgroups website`
newsgroups_train = fetch_20newsgroups(subset='train',  categories=dataset_categories, shuffle=True, random_state=42)
newsgroups_test = fetch_20newsgroups(subset='test',  categories=dataset_categories, shuffle=True, random_state=42)

## Dataset Indetification
**This is how to identify which data set to use (Please copy  the following information in report front   page).**

In [1]:
index = input('Type your student number: ')

type your student number? 28006461


In [3]:
x = int(index) % 4
y = int(index) % 3
print('This is your data set index ----> (', x, y, ')' )

This is your data set index ----> ( 1 0 )


In [19]:
data1 = newsgroups_train.target_names[x]
data2 = newsgroups_train.target_names[y]
categories = [data1, data2]
print(categories)

['comp.graphics', 'alt.atheism']


# Step 2: Process your text data, extract features

## 2.1 NLP Pre-processing
Process your text data

**An example is provided. You have modify to apply to data set based on your student number.**

In [60]:
# write your own NLP precessing examples with  preprocessing techniques.

import nltk
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)

# Get data
data = newsgroups_train.data[128]
print(data)

# Tokenise using word_tokenize
from nltk.tokenize import word_tokenize
tokenised = word_tokenize(data)
print("-------------------------tokenize:")
print(tokenised)

# Stemming  
stemmer = nltk.stem.PorterStemmer()
stemmed = stemmer.stem(data)
print("-------------------------stemming:")
print(stemmed)

From: eeerik@cc.newcastle.edu.au
Subject: Color palette for 256 color VGA rainbow
Organization: University of Newcastle, AUSTRALIA
Lines: 11

Does anybody out there have or know how to calculate the RGB values 
required to set the 256 color VGA palette so that the colors from 
0..255 will give 256 colors  of the rainbow ie red, orange, 
yellow, etc.

Any help would be appreciated. Please email to eeerik@cc.newcastle.edu.au

Erik de Castro Lopo,
Dept. Electrical & Computer Eng.,
Uni. of Newcastle,
Australia.

-------------------------tokenize:
['From', ':', 'eeerik', '@', 'cc.newcastle.edu.au', 'Subject', ':', 'Color', 'palette', 'for', '256', 'color', 'VGA', 'rainbow', 'Organization', ':', 'University', 'of', 'Newcastle', ',', 'AUSTRALIA', 'Lines', ':', '11', 'Does', 'anybody', 'out', 'there', 'have', 'or', 'know', 'how', 'to', 'calculate', 'the', 'RGB', 'values', 'required', 'to', 'set', 'the', '256', 'color', 'VGA', 'palette', 'so', 'that', 'the', 'colors', 'from', '0..255', 'will', 

## 2.2 Feature Extraction
Extract features

In [ ]:
# tf-idf: search: scikit learn tf-idf https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
# search scikit learn CountVectorizer example: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html 

# pos_taging: search: nltk pos tagging example
  # https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
pos_tagged = nltk.pos_tag(tokenised)
print("-------------------------pos_tagging:")
print(pos_tagged)

# consituency parsing, chunking
  # understand sentence grammar to know the proper sections of text.
pattern = """NP: {<DT>?<JJ>*<NN>}
             VBD: {<VBD>}
             IN: {<IN>}"""
cp = nltk.RegexpParser(pattern)
result = cp.parse(pos_tagged)
print(result)

## 2.3 Vectorisation

Convert them into vectors

**Some preprocessing are provided for convenience. Please include why NLP preprocessing is in your report. Explain what techniques have been applied in your coursework.**

In [ ]:
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
from tqdm import tqdm
from nltk.corpus import stopwords
stopwordEn = stopwords.words('english')
from nltk.corpus import wordnet
nltk.download('wordnet', quiet=True)
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

def lemmaWord(word):
    lemma = wordnet.morphy(word)
    if lemma is not None:
        return lemma
    else:
        return word

def stemWord(word):
    stem = stemmer.stem(word)
    if stem is not None:
        return stem
    else:
        return word

def processText(text, lemma=False, gram=1, rmStop=True): # default remove stop words
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b|@\w+|#', '', text, flags=re.MULTILINE) #delete URL, #hashtag# , and @xxx
    tokens = word_tokenize(text)
    whitelist = ["n't", "not", "no"]
    new_tokens = []
    stoplist = stopwordEn if rmStop else []
    for i in tokens:
      i = i.lower()
      if i.isalpha() and (i not in stoplist or i in whitelist):  #i not in ['.',',',';']  and (...)
        if lemma: i = lemmaWord(i)
        new_tokens.append(i)
    del tokens
    # tokens = [lemmaWord(i.lower()) if lemma else i.lower() for i in tokens if (i.lower() not in stoplist or i.lower() in whitelist) and i.isalpha()]
    if gram<=1:
        return new_tokens
    else:
        return [' '.join(i) for i in nltk.ngrams(new_tokens, gram)]

In [27]:
def getTags(text):
  token = word_tokenize(text)
  token = [l.lower() for l in token]
  train_tags = nltk.pos_tag(token)
  return [i[1] for i in train_tags]

In [62]:
print(processText(dataset))

['joseph', 'dale', 'fisher', 'subject', 'anger', 'organization', 'indiana', 'university', 'lines', 'article', 'writes', 'paul', 'conditt', 'writes', 'insert', 'deletion', 'paul', 'aaron', 'discourse', 'anger', 'ref', 'galatians', 'know', 'obvious', 'not', 'speaking', 'acts', 'flesh', 'speaking', 'emotions', 'emotions', 'not', 'moral', 'immoral', 'good', 'bad', 'emotions', 'first', 'step', 'not', 'label', 'emotion', 'good', 'bad', 'numb', 'hide', 'true', 'feelings', 'accept', 'god', 'accepts', 'us', 'oh', 'definitely', 'please', 'look', 'colossians', 'ephesians', 'emotions', 'controlled', 'god', 'puts', 'strong', 'emphasis', 'otherwise', 'would', 'paul', 'write', 'timothy', 'much', 'making', 'sure', 'teach', 'insert', 'deletion', 'remainder', 'paragraph', 'aaron', 'quick', 'judge', 'forgiven', 'aids', 'dealt', 'taken', 'responsibility', 'feelings', 'made', 'appropriate', 'choices', 'action', 'feelings', 'not', 'given', 'anger', 'please', 'joe', 'issue', 'especially', 'feelings', 'action

In [63]:
print(getTags(dataset))

['IN', ':', 'NN', 'VBZ', 'NN', '(', 'JJ', 'NN', 'NN', ')', 'NN', ':', 'NN', ':', 'NN', 'NN', ':', 'NN', 'NN', 'NNS', ':', 'CD', 'IN', 'NN', 'NNP', 'NN', 'NNP', 'NN', 'NNP', 'NN', 'NNP', 'NN', 'VBZ', ':', 'NN', 'NN', 'NN', 'NN', 'VBZ', ':', 'NN', 'JJ', 'NN', 'IN', 'NN', 'POS', 'CC', 'NN', 'POS', 'NN', 'IN', 'NN', ',', 'JJ', 'NNS', 'JJ', 'NNP', 'NNP', 'NNP', 'NN', 'VBP', 'RB', 'VB', 'WRB', 'PRP', 'VBZ', 'RB', 'JJ', '.', 'PRP', 'VBP', 'RB', 'VBG', 'IN', 'NNS', 'IN', 'DT', 'NNP', 'NN', '.', 'PRP', 'VBP', 'RB', 'VBG', 'IN', 'NNS', '.', 'NNS', 'VBP', 'RB', 'IN', 'PRP', 'JJ', 'JJ', 'CC', 'JJ', ',', 'JJ', 'CC', 'JJ', '.', 'NNS', 'RB', 'VBP', '.', 'DT', 'JJ', 'NN', 'VBZ', 'RB', 'RB', 'TO', 'VB', 'PRP$', 'NN', 'IN', 'JJ', 'CC', 'JJ', 'CC', 'TO', 'VB', 'PRP', 'RB', 'IN', 'NN', 'PRP', 'VBP', 'PRP$', 'JJ', 'NNS', ',', 'PRP', 'VBZ', 'TO', 'VB', 'PRP', 'IN', 'PRP', 'VBP', ',', 'IN', 'JJ', 'NN', 'NNS', 'PRP', '.', 'UH', ',', 'CC', 'PRP', 'RB', 'MD', 'VB', '.', 'VB', 'NN', 'IN', 'NNS', 'JJ', 'CC', 'VBZ

# Step 3: Pipeline

## 3.1 Construction
Modelling

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier, LogisticRegression

# Level: Lexicon. Model: tf-idf
text_clf = Pipeline([
    # add your code about text processing           
    ('vect', CountVectorizer(analyzer=processText)), 
    ('tfidf', TfidfTransformer(use_idf=True)),

    # change your classifier here, search: sklearn logistic regression example
    ('clf', SGDClassifier())
   #  ('clf', LogisticRegression())

])

# Level: Morphology

# Level: Syntax

# Level: Semantics 

## 3.2 Training
Train models on the data set (select model, tune different parameters)

In [30]:
# To train the model 
text_clf.fit(newsgroups_train.data, newsgroups_train.target)
 

Pipeline(steps=[('vect',
                 CountVectorizer(analyzer=<function processText at 0x7f54b117bdd0>)),
                ('tfidf', TfidfTransformer()), ('clf', SGDClassifier())])

# Step 4: Prediction

In [31]:
# To make prediction with dev/test set
predicted = text_clf.predict(newsgroups_test.data)

# Step 5: Evaluation

In [34]:
# To evaluate your prediction on dev set
from sklearn import metrics
print("Accuracy:", metrics.accuracy_score(newsgroups_test.target, predicted))

print(metrics.classification_report(newsgroups_test.target, predicted, target_names=newsgroups_test.target_names))

# confusion class
pd.DataFrame(metrics.confusion_matrix(newsgroups_test.target, predicted),
             columns=newsgroups_test.target_names,index=newsgroups_test.target_names)

Accuracy: 0.918774966711052
                        precision    recall  f1-score   support

           alt.atheism       0.92      0.81      0.86       319
         comp.graphics       0.94      0.96      0.95       389
               sci.med       0.94      0.94      0.94       396
soc.religion.christian       0.88      0.95      0.91       398

              accuracy                           0.92      1502
             macro avg       0.92      0.91      0.92      1502
          weighted avg       0.92      0.92      0.92      1502



,alt.atheism,comp.graphics,sci.med,soc.religion.christian
alt.atheism,257,4,11,47
comp.graphics,6,375,7,1
sci.med,6,15,371,4
soc.religion.christian,9,7,5,377


# Step 6: Error Analysis and Discussion
Analysis and discussions

Write down your own obseration about the predictions. Consider both confusion matrix and selected examples. Which classes are predicted correctly or incorrecly, possible explaination, possible solutions 

Exmaple: 1) Lab Practical, which feature is helpful for female name classification. https://www.nltk.org/book/ch06.html 
2) research paper: https://github.com/yoonkim/CNN_sentence 


In [36]:
df_pred = pd.DataFrame({'news':newsgroups_test.data,'prediction':predicted, 'true':newsgroups_test.target})
df_pred[df_pred['true'] != df_pred['prediction']]

,news,prediction,true
12,"From: ""Gabriel D. Underwood"" <gabe+@CMU.EDU>\n...",3,2
15,From: swf@elsegundoca.ncr.com (Stan Friesen)\n...,2,0
19,From: mathew <mathew@mantis.co.uk>\nSubject: D...,3,0
26,From: UC512052@mizzou1.missouri.edu (David K. ...,2,1
36,Organization: Penn State University\nFrom: <RF...,0,2
...,...,...,...
1438,From: mangoe@cs.umd.edu (Charley Wingate)\nSub...,0,3
1450,From: alan.barclay@almac.co.uk (Alan Barclay)\...,1,2
1455,From: pww@spacsun.rice.edu (Peter Walker)\nSub...,3,0
1480,From: wilsonr@logica.co.uk\nSubject: Re: What ...,3,0


# References

https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html 

https://www.nltk.org/book/ch06.html 

search: Other online resources: 

https://towardsdatascience.com/setting-up-text-preprocessing-pipeline-using-scikit-learn-and-spacy-e09b9b76758f 
 
sentiment analysis scikit learn 

scikit learn or nltk + NLP techniques 

python + NLP techniques

scikit learn logistic regression


